In [ ]:
# Check the API & News and print out inital web values
import os
import requests
import sqlite3
from datetime import datetime, timezone

# -----------------------------------------------------------------------------
# Configuration
# -----------------------------------------------------------------------------
# 1) If you set NEWSAPI_KEY as an environment variable, use that:
API_KEY = os.getenv("NEWSAPI_KEY", "5d4138ab664242dd9f5cdd118fb174b4")

# 2) Choose which endpoint and parameters you want. Here: Top Headlines in English, global:
NEWSAPI_URL = "https://newsapi.org/v2/top-headlines"
PARAMS = {
    "language": "en",
    "pageSize": 20,       # up to 100 max on free tier
    # You can also filter by country (e.g. 'us') or category (e.g. 'business'):
    # "country": "us",
    # "category": "business",
    "apiKey": API_KEY
}

# 3) (Optional) SQLite settings—uncomment if you want to save into SQLite
DB_PATH = "newsapi_headlines.db"

# -----------------------------------------------------------------------------
# 1. Fetch Top Headlines from NewsAPI
# -----------------------------------------------------------------------------
def fetch_top_headlines():
    """
    Sends a GET request to NewsAPI’s /v2/top-headlines endpoint
    and returns the JSON “articles” array.
    """
    resp = requests.get(NEWSAPI_URL, params=PARAMS, timeout=10)
    resp.raise_for_status()
    data = resp.json()

    if data.get("status") != "ok":
        raise RuntimeError(f"NewsAPI returned error: {data.get('code')} - {data.get('message')}")

    articles = data.get("articles", [])
    print(f"→ Retrieved {len(articles)} articles from NewsAPI")
    return articles

# -----------------------------------------------------------------------------
# 2. (Optional) Initialize SQLite for storing headlines
# -----------------------------------------------------------------------------
def init_sqlite_db(db_path=DB_PATH):
    """
    Create a simple SQLite database with a table to hold NewsAPI results.
    Fields:
      - article_id     (TEXT, PRIMARY KEY)       → we'll derive from URL
      - url            (TEXT)
      - headline       (TEXT)
      - author         (TEXT)
      - source_name    (TEXT)
      - publish_ts     (TEXT)                    → ISO timestamp
      - description    (TEXT)
      - content        (TEXT)
      - crawl_ts       (TEXT)                    → when we fetched it
    """
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS newsapi_headlines (
            article_id    TEXT PRIMARY KEY,
            url           TEXT NOT NULL,
            headline      TEXT NOT NULL,
            author        TEXT,
            source_name   TEXT,
            publish_ts    TEXT NOT NULL,
            description   TEXT,
            content       TEXT,
            crawl_ts      TEXT NOT NULL
        );
    """)
    conn.commit()
    return conn

# -----------------------------------------------------------------------------
# 3. Save NewsAPI articles into SQLite
# -----------------------------------------------------------------------------
def save_to_sqlite(conn, articles):
    """
    Insert or ignore each article into the SQLite table.
    We derive article_id by hashing the URL (or by simply using the URL).
    """
    import hashlib

    cur = conn.cursor()
    insert_sql = """
      INSERT OR IGNORE INTO newsapi_headlines (
          article_id, url, headline, author, source_name,
          publish_ts, description, content, crawl_ts
      ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);
    """

    crawl_ts = datetime.now(timezone.utc).isoformat()
    for art in articles:
        url = art.get("url", "").strip()
        # Derive a simple article_id using SHA1 of the URL
        article_id = hashlib.sha1(url.encode("utf-8")).hexdigest()

        headline = art.get("title", "").strip()
        author = art.get("author", None)
        source_name = art.get("source", {}).get("name", None)
        publish_str = art.get("publishedAt", None)  # e.g. "2025-06-05T13:36:00Z"
        # If publishAt is missing, skip this article
        if not publish_str:
            continue

        description = art.get("description", None)
        content = art.get("content", None)

        cur.execute(insert_sql, (
            article_id,
            url,
            headline,
            author,
            source_name,
            publish_str,
            description,
            content,
            crawl_ts
        ))

    conn.commit()
    print(f"→ Saved {len(articles)} articles to SQLite (duplicates ignored).")

# -----------------------------------------------------------------------------
# 4. Main: Fetch & (Optionally) Save
# -----------------------------------------------------------------------------
def main():
    # Check API key
    if not API_KEY or API_KEY == "YOUR_API_KEY_HERE":
        print("Error: You must set your NewsAPI key. Either define NEWSAPI_KEY env var or edit this script.")
        return

    # Step 1: Fetch top headlines
    articles = fetch_top_headlines()

    # Print out each headline + URL
    for idx, art in enumerate(articles, start=1):
        title = art.get("title", "").strip()
        url = art.get("url", "").strip()
        pub = art.get("publishedAt", "")
        source = art.get("source", {}).get("name", "")
        print(f"{idx}. [{source}] {title}")
        print(f"   → {url}")
        print(f"   Published: {pub}\n")

    # Step 2: (Optional) Save to SQLite
    conn = init_sqlite_db()
    save_to_sqlite(conn, articles)
    conn.close()

if __name__ == "__main__":
    main()

→ Retrieved 16 articles from NewsAPI
1. [The Washington Post] Edmund White, acclaimed novelist of gay life, dies at 85 - The Washington Post
   → https://www.washingtonpost.com/obituaries/2025/06/04/edmund-white-dead-novelist/
   Published: 2025-06-04T16:31:08Z

2. [The Washington Post] German city evacuates 20,000 after unexploded World War II bombs are found - The Washington Post
   → https://www.washingtonpost.com/world/2025/06/04/cologne-germany-bombs/
   Published: 2025-06-04T16:00:40Z

3. [New York Post] Jay Wright colleague reveals former Villanova coach’s chances of taking Knicks job - New York Post
   → https://nypost.com/2025/06/04/sports/jay-wright-colleague-reveals-ex-villanova-coachs-chances-of-taking-knicks-job/
   Published: 2025-06-04T15:19:00Z

4. [BBC News] Eleven people die in crush at Bengaluru IPL victory parade - BBC
   → https://www.bbc.com/news/articles/cdr5mrrj1ego
   Published: 2025-06-04T15:04:02Z

5. [Investor's Business Daily] Stock Market Today: Indexes Up

In [ ]:
# Parse news data based on category and add sentiment score, save into database
import os
import requests
import sqlite3
import hashlib
import nltk
from datetime import datetime, timezone

# ─────────────────────────────────────────────────────────────────────────────
# 1. Ensure VADER lexicon is downloaded
# ─────────────────────────────────────────────────────────────────────────────
try:
    nltk.data.find("sentiment/vader_lexicon.zip/vader_lexicon/vader_lexicon.txt")
except LookupError:
    nltk.download("vader_lexicon")

from nltk.sentiment.vader import SentimentIntensityAnalyzer

# -----------------------------------------------------------------------------
# 2. Configuration
# -----------------------------------------------------------------------------
API_KEY = os.getenv("NEWSAPI_KEY", "5d4138ab664242dd9f5cdd118fb174b4")
TOP_HEADLINES_URL = "https://newsapi.org/v2/top-headlines"
EVERYTHING_URL    = "https://newsapi.org/v2/everything"

CATEGORIES = ["business", "health", "science", "technology"]
KEYWORDS = [
    "\"record high\"",
    "\"record low\"",
    "stocks",
    "bonds",
    "forex",
    "economic",
    "trump",
    "china",
    "\"new tech\""
]

COMMON_PARAMS = {
    "language": "en",
    "pageSize": 20,
    "apiKey": API_KEY
}

DB_PATH = "newsapi_global.db"

# -----------------------------------------------------------------------------
# 3. Initialize / Connect to SQLite
#    - Creates `newsapi_articles` table if needed
#    - Ensures `sentiment_score` column exists (via ALTER TABLE if necessary)
# -----------------------------------------------------------------------------
def init_sqlite_db(db_path=DB_PATH):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    # 3.1 Create table if it doesn't exist (without sentiment_score)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS newsapi_articles (
            article_id      TEXT      PRIMARY KEY,
            url             TEXT      NOT NULL,
            headline        TEXT      NOT NULL,
            author          TEXT,
            source_name     TEXT,
            publish_ts      TEXT      NOT NULL,
            description     TEXT,
            content         TEXT,
            crawl_ts        TEXT      NOT NULL,
            category        TEXT,
            query           TEXT
            -- sentiment_score column will be added below if missing
        );
    """)
    conn.commit()

    # 3.2 Check if sentiment_score column already exists
    cur.execute("PRAGMA table_info(newsapi_articles);")
    columns = [info[1] for info in cur.fetchall()]

    # If sentiment_score isn't present, add it now
    if "sentiment_score" not in columns:
        cur.execute("ALTER TABLE newsapi_articles ADD COLUMN sentiment_score REAL;")
        conn.commit()

    return conn

# -----------------------------------------------------------------------------
# 4. Helper: Generate unique article_id from URL
# -----------------------------------------------------------------------------
def make_article_id(url: str) -> str:
    return hashlib.sha1(url.encode("utf-8")).hexdigest()

# -----------------------------------------------------------------------------
# 5. Fetch top-headlines by category
# -----------------------------------------------------------------------------
def fetch_top_headlines_by_category(category: str):
    params = COMMON_PARAMS.copy()
    params["category"] = category

    resp = requests.get(TOP_HEADLINES_URL, params=params, timeout=10)
    resp.raise_for_status()
    data = resp.json()

    if data.get("status") != "ok":
        raise RuntimeError(f"NewsAPI top-headlines error: {data.get('code')} - {data.get('message')}")

    out = []
    for item in data.get("articles", []):
        url = item.get("url", "").strip()
        if not url:
            continue

        out.append({
            "url": url,
            "headline": item.get("title", "").strip(),
            "author": item.get("author"),
            "source_name": item["source"].get("name"),
            "publish_ts": item.get("publishedAt"),
            "description": item.get("description"),
            "content": item.get("content"),
            "category": category,
            "query": None
        })
    return out

# -----------------------------------------------------------------------------
# 6. Fetch “everything” by keyword
# -----------------------------------------------------------------------------
def fetch_everything_by_keyword(keyword: str):
    params = COMMON_PARAMS.copy()
    params["q"] = keyword
    params["sortBy"] = "publishedAt"

    resp = requests.get(EVERYTHING_URL, params=params, timeout=10)
    resp.raise_for_status()
    data = resp.json()

    if data.get("status") != "ok":
        raise RuntimeError(f"NewsAPI everything error: {data.get('code')} - {data.get('message')}")

    out = []
    for item in data.get("articles", []):
        url = item.get("url", "").strip()
        if not url:
            continue

        out.append({
            "url": url,
            "headline": item.get("title", "").strip(),
            "author": item.get("author"),
            "source_name": item["source"].get("name"),
            "publish_ts": item.get("publishedAt"),
            "description": item.get("description"),
            "content": item.get("content"),
            "category": None,
            "query": keyword
        })
    return out

# -----------------------------------------------------------------------------
# 7. Save a batch of articles to SQLite (including sentiment_score)
# -----------------------------------------------------------------------------
def save_to_sqlite(conn, articles, sia: SentimentIntensityAnalyzer):
    cur = conn.cursor()

    insert_sql = """
      INSERT OR IGNORE INTO newsapi_articles (
          article_id, url, headline, author, source_name,
          publish_ts, description, content, crawl_ts, category, query,
          sentiment_score
      ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
    """

    crawl_ts = datetime.now(timezone.utc).isoformat()

    for art in articles:
        article_id = make_article_id(art["url"])

        # Combine headline + description (if exists) for sentiment
        text_to_score = art["headline"]
        if art["description"]:
            text_to_score += " " + art["description"]

        # Compute VADER compound sentiment (–1.0 to +1.0)
        sentiment = sia.polarity_scores(text_to_score)["compound"]

        vals = (
            article_id,
            art["url"],
            art["headline"],
            art["author"],
            art["source_name"],
            art["publish_ts"],
            art["description"],
            art["content"],
            crawl_ts,
            art["category"],
            art["query"],
            sentiment
        )
        cur.execute(insert_sql, vals)

    conn.commit()

# -----------------------------------------------------------------------------
# 8. One‐Shot Fetch & Save (including sentiment)
# -----------------------------------------------------------------------------
def run_once():
    if not API_KEY or "YOUR_API_KEY_HERE" in API_KEY:
        print("Error: set your NewsAPI key (either in NEWSAPI_KEY env or hard-coded).")
        return

    # 8.1 Initialize DB (create table if needed, then ALTER to add sentiment_score)
    conn = init_sqlite_db()

    # 8.2 Initialize VADER sentiment analyzer
    sia = SentimentIntensityAnalyzer()

    all_articles = []

    # 8.3 Fetch top-headlines for each category
    for cat in CATEGORIES:
        try:
            cat_list = fetch_top_headlines_by_category(cat)
            print(f"Fetched {len(cat_list)} headlines in '{cat}'")
            all_articles.extend(cat_list)
        except Exception as e:
            print(f"Error fetching category '{cat}': {e!s}")

    # 8.4 Fetch “everything” for each keyword
    for kw in KEYWORDS:
        try:
            kw_list = fetch_everything_by_keyword(kw)
            print(f"Fetched {len(kw_list)} everything‐results for '{kw}'")
            all_articles.extend(kw_list)
        except Exception as e:
            print(f"Error fetching keyword '{kw}': {e!s}")

    # 8.5 Save everything into SQLite (sentiment computed here)
    save_to_sqlite(conn, all_articles, sia)
    print(f"Saved {len(all_articles)} articles to SQLite (duplicates ignored).")

    conn.close()

if __name__ == "__main__":
    run_once()

Fetched 17 headlines in 'business'
Fetched 18 headlines in 'health'
Fetched 20 headlines in 'science'
Fetched 18 headlines in 'technology'
Fetched 19 everything‐results for '"record high"'
Fetched 17 everything‐results for '"record low"'
Fetched 20 everything‐results for 'stocks'
Fetched 20 everything‐results for 'bonds'
Fetched 20 everything‐results for 'forex'
Fetched 20 everything‐results for 'economic'
Fetched 18 everything‐results for 'trump'
Fetched 20 everything‐results for 'china'
Fetched 20 everything‐results for '"new tech"'
Saved 247 articles to SQLite (duplicates ignored).


In [ ]:
# Get News data from Database and save into csv file
import sqlite3
import pandas as pd

# 1) Connect to the SQLite database
conn = sqlite3.connect("newsapi_global.db")

# 2) List all tables
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Available tables:\n", tables, "\n")

# 3) Read first 10 rows of newsapi_articles (if it exists)
if "newsapi_articles" in tables["name"].values:
    df = pd.read_sql_query("SELECT * FROM newsapi_articles LIMIT 10;", conn)
    print("First 10 rows of newsapi_articles:\n")
    print(df.to_string(index=False))  # print as plain text table

    # 4) Export the entire table to CSV
    df_all = pd.read_sql_query("SELECT * FROM newsapi_articles;", conn)
    csv_path = r"C:\Users\Jerry Hunter\Downloads\DB_NewsV2.csv"
    df_all.to_csv(csv_path, index=False)
    print(f"\nExported full table to: {csv_path}")
else:
    print("Table 'newsapi_articles' does not exist.")

conn.close()

Available tables:
                name
0  newsapi_articles 

First 10 rows of newsapi_articles:

                              article_id                                                                                                                                                                    url                                                                                                                                   headline                    author               source_name           publish_ts                                                                                                                                                       description                                                                                                                                                                                                                  content                         crawl_ts category query sentiment_score
5acc4048caa54591f06a313ade11d34545a847c6

In [ ]:
#Read csv file and print out by rank of sentiment socre of news
import pandas as pd

# 1. Path to your CSV file
csv_path = r"C:\Users\Jerry Hunter\Downloads\DB_NewsV2.csv"

# 2. Load the CSV into a DataFrame
df = pd.read_csv(csv_path)

# 3. Verify the 'sentiment_score' column exists
if "sentiment_score" not in df.columns:
    raise ValueError("The CSV does not contain a 'sentiment_score' column.")

# 4. Sort by sentiment_score (highest first) and take top 10
top10 = df.sort_values(by="sentiment_score", ascending=False).head(10)

# 5. Drop the columns you don’t want to display
to_hide = ["article_id", "url", "author"]
display_df = top10.drop(columns=[c for c in to_hide if c in top10.columns])

# 6. Print the top 10 rows without those columns
print("Top 10 articles by sentiment_score (high to low):\n")
print(display_df.to_string(index=False))

Top 10 articles by sentiment_score (high to low):

                                                                                                                  headline             source_name           publish_ts                                                                                                                                                                                                                                                                description                                                                                                                                                                                                                      content                         crawl_ts category         query  sentiment_score
                                                           These 5 F&O stocks saw a high increase in futures open interest       Economictimes.com 2025-06-05T03:35:51Z       As of June 4, there was a notable surge in fut